In [67]:
%reload_ext autoreload
%autoreload 2
import numpy as np
from qutip import *
from qutip.qip.device import *
from qutip.qip.operations import *
from qutip.qip.circuit import QubitCircuit
from cvqaoa import *
from cvqaoa.gates import carb

In [68]:
def product(*args, repeat=1):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in pools:
        result = [x + [y] for x in result for y in pool]
    for prod in result:
        yield list(prod)

In [70]:
file = np.load('../data/kraus/cv_kraus_rx.npz', allow_pickle=True, fix_imports=True)

In [74]:
kraus = file['kraus']
sum(k@np.conj(k.T) for k in kraus[1])

array([[9.99998098e-01+0.00000000e+00j, 5.06321452e-07-3.62225699e-19j],
       [5.06321452e-07+3.62225699e-19j, 9.99998098e-01+0.00000000e+00j]])

In [3]:
N = 2  # qubits
d = 4  # dimension
tau = 2  # gate time
# Load the average gate fidelity
file = np.load('../data/average_gate_fidelity/cv_avg_fid_zz.npz')
f_bar = np.mean(file['avg'])
# Find the corresponding T1
gamma = 2 * (d + 1) / (d * tau) * (1 - f_bar)
T1 = 1 / (gamma)

# Pauli matrices
P = [qeye(2), sigmax(), sigmay(), sigmaz()]

# Create all tensorproducts
sigma = list(map(tensor, product(P, repeat=2)))

# Initialize qubit processor
qp = QubitProcessor(N=N, T1=T1)

# Quantum circuit
qc = QubitCircuit(N)
qc.user_gates = {"CARB": carb}
qc.add_gate("CARB", targets=[0, 1], arg_value=0)

In [4]:
# Initialize PTM
R = np.zeros((d**2, d**2))

for j in range(d**2):
    result = qp.run_state(init_state=sigma[j], qc=qc)
    # Channel
    Lambda = result.states[-1]
    for i in range(d**2):
        R[i, j] = 1 / d * (sigma[i] * Lambda).tr().real

In [5]:
# Convert PTM to Choi-matrix
choi = 1 / d**2 * sum(R[i, j] * tensor(sigma[j].trans(), sigma[i]) for i in range(d**2) for j in range(d**2))
choi.dims = [[[2, 2], [2, 2]],[[2, 2], [2, 2]]]
choi.superrep = 'choi'

In [7]:
# Find eigenvectors and eigenvalues to choi
eValues, eVectors = np.linalg.eigh(choi.full())
# Because of machine impressision we drop terms smaller than rtol
rtol = 1e-7
idx, = np.where(eValues<rtol)
eVectors = np.delete(eVectors, idx, axis=1) # drop columns
eValues = np.delete(eValues, idx)
num = len(eValues)
# Get the Kraus operators
kraus = [np.sqrt(d*eValues[i])*eVectors[:,i].reshape((d,d)) for i in range(num)]
# Check that they sum to 1
np.isclose(Qobj(sum(k@np.conj(k.T) for k in kraus)),qeye(4)).all()

True

In [142]:
# Convert Choi to chi
chi = choi_to_chi(choi)

In [ ]:
A[i,j,m,n] = (sigma[i]*sigma[m]*sigma[j]*sigma[n]).tr()

In [157]:
A = np.zeros((16,16,16,16),dtype='complex')
for i in range(16):
    for j in range(16):
        for m in range(16):
            for n in range(16):
                A[i,j,m,n] = (sigma[i]*sigma[m]*sigma[j]*sigma[n]).tr()

In [158]:
A = A.reshape((16**2,16**2))
A_inv = np.linalg.inv(A)

In [171]:
R_vec = R.reshape((16**2,1))
chi_vec = d*A_inv@R_vec

In [187]:
np.isclose(Qobj(chi_vec.reshape(16,16)).trans(), chi/4).all()

True

In [242]:
w, v = np.linalg.eigh(chi.full())

In [145]:
# Qobj(v@np.diag(w)@np.conj(v.T)).tidyup()

In [202]:
e = w[w > 0]

In [215]:
kraus = [np.sqrt(w[i])*sum(v[j,i]*sigma[j].full() for j in range(d**2)) for i in range(len(e))]

/var/folders/b0/ymj78yz13kz0l30_vjntj2cc0000gn/T/ipykernel_4521/4182599287.py:1: RuntimeWarning: invalid value encountered in sqrt
  kraus = [np.sqrt(w[i])*sum(v[j,i]*sigma[j].full() for j in range(d**2)) for i in range(len(e))]


In [225]:
kraus = [np.sqrt(np.round(w[i],6))*sum(v[j,i]*sigma[j].full() for j in range(d**2)) for i in range(len(w))]

In [232]:
i = sum(k@np.conj(k.T) for k in kraus) / d
Qobj(i).tidyup()

Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.99999993 0.         0.         0.        ]
 [0.         0.99999998 0.         0.        ]
 [0.         0.         0.99999998 0.        ]
 [0.         0.         0.         1.0000001 ]]

In [188]:
kraus = choi_to_kraus(choi)
i = sum(k*k.dag() for k in kraus)*4